In [1]:
# parsing XML annotations manuelles

In [59]:
from lxml import etree

# récupérer les annotations manuelles sur un fichier au format "Inline XML"
def recup_annotations_manuelles(fichier):
    annotations = []
    parser = etree.XMLParser(recover=True)
    root = etree.parse(fichier, parser)
    balise_EN = "de.tudarmstadt.ukp.dkpro.core.api.ner.type.NamedEntity"
    
    for el in root.iter(balise_EN):
        annotations.append((int(el.attrib["begin"]), int(el.attrib["end"]), el.attrib["value"]))
    return annotations

# récupérer le texte d'un fichier ".txt"
def recup_texte(fichier):
    with open(fichier, "r", encoding="utf-8") as f:
        texte = f.read()
    return texte

In [64]:
chemin_fichier = "Evaluation/5408048.xml"
fichier_text = "%s.txt" % chemin_fichier
fichier_xml = "%s.xml" % chemin_fichier

print(len(recup_annotations(fichier_xml)))
print(len(recup_texte(fichier_text)))
#print(recup_annotations(fichier))

133
95439


In [83]:
import csv

chemin_fichier_spacy = "Evaluation/5431000_PER_2.csv"

import csv
with open(chemin_fichier_spacy, "r", encoding="utf-8", newline="") as csvfile:
    reader = csv.DictReader(csvfile, delimiter=",")
    for row in reader:
        print(row['Tag'], row['Indice_Début'], row['Indice_Fin'])

PER 57 80
PER 177 197
PER 266 275
PER 1203 1207
PER 2095 2103
PER 2542 2550
PER 2554 2559
PER 8621 8629
PER 11871 11880
PER 27298 27309
PER 27370 27375
PER 28914 28921
PER 30803 30811
PER 31194 31202
PER 34973 34977
PER 35025 35029
PER 35125 35129
PER 35987 36000
PER 36010 36012
PER 36748 36754
PER 41595 41610
PER 42745 42753
PER 44304 44312
PER 44366 44374
PER 44572 44580
PER 46759 46785
PER 53646 53657
PER 53874 53880
PER 55299 55310
PER 55957 55963
PER 57886 57890
PER 57931 57935
PER 58110 58119
PER 60201 60212
PER 60797 60806
PER 62902 62910
PER 62931 62943
PER 63975 63977
PER 69051 69056
PER 71622 71631
PER 73282 73283
PER 76901 76906
PER 85465 85466
PER 91917 91924
PER 98545 98547
PER 101006 101013
PER 103666 103674
PER 103941 103943
PER 104818 104832
PER 109298 109314
PER 109334 109357
PER 109444 109452
PER 109547 109555
PER 109664 109671
PER 118965 118972
PER 119543 119553
PER 119841 119849
PER 119900 119910
PER 120365 120385
PER 122458 122467
PER 123506 123514
PER 124414 12443